# 3D Anderson Model Mobility Edge Analysis

This notebook reproduces the plots from `analysis/3dAnderson-mobility-analysis.py` for interactive exploration and debugging.

**Figures:**
1. DOS and IPR Summary (2x3 grid)
2. Energy-Resolved IPR (mobility edge visualization)
3. Mobility Edge Trajectory vs disorder
4. Filtered r/z Statistics

## 1. Imports and Configuration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import glob

# Plot style constants
COLORS = {'H': 'blue', 'SL': 'orange'}
FIGSIZE_2x3 = (18, 10)
FIGSIZE_2x2 = (18, 18)
TITLE_SIZE = 20
LABEL_SIZE = 20
SUPTITLE_SIZE = 24
ANNOTATION_PROPS = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Reference values for spectral statistics
GOE_R = 0.5295
POISSON_R = 0.386
GOE_Z = 0.5687
POISSON_Z = 0.5

# IPR percentile cutoff for energy-resolved plots
IPR_PERCENTILE_CUTOFF = 90

# Data directory
DATA_DIR = '../data/'

## 2. Data Loading Utilities

In [ ]:
def parse_filename(filepath):
    """
    Parse a 3D Anderson data filename to extract parameters.
    Expected format: 3dAnderson_L{L}_disorder{start}-{end}_numEigs{n}_realizations{r}_{date}_*.dat
    """
    basename = os.path.basename(filepath)
    params = {}
    
    # Extract L
    match = re.search(r'_L(\d+)_', basename)
    if match:
        params['L'] = int(match.group(1))
    
    # Extract disorder range
    match = re.search(r'_disorder([\d.]+)-([\d.]+)_', basename)
    if match:
        params['disorder_start'] = float(match.group(1))
        params['disorder_end'] = float(match.group(2))
    
    # Extract number of realizations
    match = re.search(r'_realizations(\d+)_', basename)
    if match:
        params['num_realizations'] = int(match.group(1))
    
    return params


def find_3d_anderson_files(data_dir, L=None):
    """
    Find all 3D Anderson data files in the given directory.
    Returns dict mapping base names to file paths by type.
    """
    pattern = os.path.join(data_dir, '3dAnderson_L*_disorder*_*.dat')
    all_files = glob.glob(pattern)
    
    # Group files by base name (everything before the file type suffix)
    file_groups = {}
    
    for filepath in all_files:
        basename = os.path.basename(filepath)
        
        # Determine the base name (strip the type suffix)
        for suffix in ['_H_eigval.dat', '_H_eigvec.dat', '_H_IPR.dat',
                       '_spectral_localiser_eigval.dat', '_spectral_localiser_eigvec.dat',
                       '_spectral_localiser_IPR.dat', '_parameters.txt', '_seeds.dat']:
            if basename.endswith(suffix):
                base = basename[:-len(suffix)]
                file_type = suffix[1:-4] if suffix.endswith('.dat') else suffix[1:-4]
                
                if base not in file_groups:
                    file_groups[base] = {'params': parse_filename(filepath), 'files': {}}
                
                # Clean up file type name
                if suffix == '_H_eigval.dat':
                    file_groups[base]['files']['H_eigval'] = filepath
                elif suffix == '_H_IPR.dat':
                    file_groups[base]['files']['H_IPR'] = filepath
                elif suffix == '_spectral_localiser_eigval.dat':
                    file_groups[base]['files']['SL_eigval'] = filepath
                elif suffix == '_spectral_localiser_IPR.dat':
                    file_groups[base]['files']['SL_IPR'] = filepath
                elif suffix == '_parameters.txt':
                    file_groups[base]['files']['parameters'] = filepath
                break
    
    # Filter by L if specified
    if L is not None:
        file_groups = {k: v for k, v in file_groups.items() if v['params'].get('L') == L}
    
    return file_groups


def infer_shape_from_file(filepath, num_realizations, basis_size, dtype='float64'):
    """
    Infer array shape from file size.
    Assumes shape is (disorder_resolution, num_realizations, num_eigs).
    """
    bytes_per_element = 8 if dtype == 'float64' else 16  # complex128
    file_size = os.path.getsize(filepath)
    total_elements = file_size // bytes_per_element
    
    # Try to infer disorder_resolution assuming num_eigs = basis_size
    if total_elements % (num_realizations * basis_size) == 0:
        disorder_resolution = total_elements // (num_realizations * basis_size)
        num_eigs = basis_size
    else:
        # Try different disorder resolutions
        for dr in range(1, 100):
            if total_elements % (dr * num_realizations) == 0:
                possible_num_eigs = total_elements // (dr * num_realizations)
                if possible_num_eigs <= basis_size:
                    disorder_resolution = dr
                    num_eigs = possible_num_eigs
                    break
        else:
            raise ValueError(f"Cannot infer shape from file size {file_size}")
    
    return (disorder_resolution, num_realizations, num_eigs)


print("Data loading utilities defined.")

## 3. Load Data

**IMPORTANT**: Set your parameters here. The shape is inferred from each file independently.

In [ ]:
# ============================================================================
# USER CONFIGURATION - Modify these values to match your data
# ============================================================================

# System size
L = 8

# Number of disorder realizations
num_realizations = 100

# Disorder range
disorder_start = 0.0
disorder_end = 30.0

# Basis sizes (for 3D Anderson)
H_basis_size = L ** 3        # 512 for L=8
SL_basis_size = 4 * L ** 3   # 2048 for L=8

# ============================================================================
# Find and load data files
# ============================================================================

file_groups = find_3d_anderson_files(DATA_DIR, L=L)
print(f"Found {len(file_groups)} data set(s) for L={L}:")
for name, info in file_groups.items():
    print(f"  {name}")
    print(f"    Files: {list(info['files'].keys())}")

In [ ]:
# Select which dataset to use (if multiple exist)
# You can change this to select a different dataset
dataset_name = list(file_groups.keys())[0] if file_groups else None

if dataset_name is None:
    raise ValueError("No data files found!")

files = file_groups[dataset_name]['files']
print(f"\nUsing dataset: {dataset_name}")
print(f"Available files: {list(files.keys())}")

In [ ]:
# Load data with shape inference from each file
data = {}

# Load H eigenvalues
if 'H_eigval' in files:
    shape = infer_shape_from_file(files['H_eigval'], num_realizations, H_basis_size)
    data['H_eigval'] = np.memmap(files['H_eigval'], dtype='float64', mode='r', shape=shape)
    print(f"H_eigval: shape {shape}, range [{data['H_eigval'].min():.4f}, {data['H_eigval'].max():.4f}]")
    disorder_resolution = shape[0]

# Load H IPR (infer shape independently!)
if 'H_IPR' in files:
    shape = infer_shape_from_file(files['H_IPR'], num_realizations, H_basis_size)
    data['H_IPR'] = np.memmap(files['H_IPR'], dtype='float64', mode='r', shape=shape)
    print(f"H_IPR: shape {shape}, range [{data['H_IPR'].min():.6f}, {data['H_IPR'].max():.6f}]")
    print(f"  Mean: {data['H_IPR'].mean():.6f}, Zeros: {np.sum(data['H_IPR'] == 0)}")
    disorder_resolution = shape[0]

# Load SL eigenvalues
if 'SL_eigval' in files:
    shape = infer_shape_from_file(files['SL_eigval'], num_realizations, SL_basis_size)
    data['SL_eigval'] = np.memmap(files['SL_eigval'], dtype='float64', mode='r', shape=shape)
    print(f"SL_eigval: shape {shape}, range [{data['SL_eigval'].min():.4f}, {data['SL_eigval'].max():.4f}]")

# Load SL IPR (infer shape independently!)
if 'SL_IPR' in files:
    shape = infer_shape_from_file(files['SL_IPR'], num_realizations, SL_basis_size)
    data['SL_IPR'] = np.memmap(files['SL_IPR'], dtype='float64', mode='r', shape=shape)
    print(f"SL_IPR: shape {shape}, range [{data['SL_IPR'].min():.6f}, {data['SL_IPR'].max():.6f}]")
    print(f"  Mean: {data['SL_IPR'].mean():.6f}, Zeros: {np.sum(data['SL_IPR'] == 0)}")

# Create disorder values array
disorder_values = np.linspace(disorder_start, disorder_end, disorder_resolution)
print(f"\nDisorder values: {disorder_resolution} points from {disorder_start} to {disorder_end}")

## 4. Helper Functions

In [ ]:
def compute_ipr(eigvec):
    """Compute IPR from eigenvectors. IPR = sum(|psi_i|^4)"""
    return np.sum(np.abs(eigvec) ** 4, axis=-1)


def calculate_r(eigval):
    """Calculate the adjacent gap ratio r."""
    eigval_sorted = np.sort(eigval)
    spacings = np.diff(eigval_sorted)
    
    min_vals = np.minimum(spacings[:-1], spacings[1:])
    max_vals = np.maximum(spacings[:-1], spacings[1:])
    
    r = np.divide(min_vals, max_vals, out=np.zeros_like(min_vals), where=max_vals != 0)
    return r.mean()


def calculate_z(eigval):
    """Calculate the next-nearest neighbor ratio z."""
    eigval_sorted = np.sort(eigval)
    s = np.diff(eigval_sorted)
    
    if len(s) < 5:
        return np.nan
    
    s_i_minus_2 = s[:-4]
    s_i_minus_1 = s[1:-3]
    s_i = s[2:-2]
    s_i_plus_1 = s[3:-1]
    
    nn = np.minimum(s_i, s_i_minus_1)
    n_other = np.maximum(s_i, s_i_minus_1)
    nnn_left = s_i_minus_1 + s_i_minus_2
    nnn_right = s_i + s_i_plus_1
    
    nnn = np.minimum.reduce([n_other, nnn_left, nnn_right])
    
    z = np.divide(nn, nnn, out=np.zeros_like(nn), where=nnn != 0)
    return z.mean()


def extract_mobility_edge(eigval, ipr, num_bins=50, ipr_threshold=None):
    """
    Extract the mobility edge from energy-resolved IPR data.
    Returns E_c_lower, E_c_upper, bin_centers, bin_ipr
    """
    if ipr_threshold is None:
        ipr_threshold = 2.0 / len(eigval)
    
    E_min, E_max = eigval.min(), eigval.max()
    bin_edges = np.linspace(E_min, E_max, num_bins + 1)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
    
    bin_ipr = np.zeros(num_bins)
    bin_counts = np.zeros(num_bins)
    
    bin_indices = np.digitize(eigval, bin_edges) - 1
    bin_indices = np.clip(bin_indices, 0, num_bins - 1)
    
    for i in range(len(eigval)):
        bin_idx = bin_indices[i]
        bin_ipr[bin_idx] += ipr[i]
        bin_counts[bin_idx] += 1
    
    bin_ipr = np.divide(bin_ipr, bin_counts, out=np.zeros_like(bin_ipr), where=bin_counts > 0)
    
    center_idx = num_bins // 2
    E_c_lower = E_min
    E_c_upper = E_max
    
    for i in range(center_idx, -1, -1):
        if bin_counts[i] > 0 and bin_ipr[i] > ipr_threshold:
            E_c_lower = bin_centers[i]
            break
    
    for i in range(center_idx, num_bins):
        if bin_counts[i] > 0 and bin_ipr[i] > ipr_threshold:
            E_c_upper = bin_centers[i]
            break
    
    return E_c_lower, E_c_upper, bin_centers, bin_ipr


def filter_eigenvalues_by_energy(eigval, E_min, E_max):
    """Filter eigenvalues to those within the energy window."""
    mask = (eigval >= E_min) & (eigval <= E_max)
    return eigval[mask]


print("Helper functions defined.")

## 5. Figure 1: DOS and IPR Summary

2x3 grid showing DOS, eigenvalues, and IPR for a single disorder value.

In [ ]:
# Select disorder index and realization
disorder_idx = disorder_resolution // 2  # Middle disorder value
realization_idx = 0

W = disorder_values[disorder_idx]
print(f"Plotting for disorder W={W:.1f}, realization {realization_idx}")

In [ ]:
# Check what data is available
has_H_eigval = 'H_eigval' in data
has_SL_eigval = 'SL_eigval' in data
has_H_IPR = 'H_IPR' in data
has_SL_IPR = 'SL_IPR' in data

# Determine grid size
n_cols = 2 if (has_H_IPR or has_SL_IPR) else 2
if has_H_IPR or has_SL_IPR:
    n_cols = 3

fig, axs = plt.subplots(2, n_cols, figsize=FIGSIZE_2x3, constrained_layout=True)

# Row 0: Hamiltonian
if has_H_eigval:
    H_eigval = data['H_eigval'][disorder_idx, realization_idx]
    
    # DOS histogram
    axs[0, 0].hist(H_eigval, bins=100, density=True, orientation='horizontal',
                   color=COLORS['H'], alpha=0.8)
    axs[0, 0].set_title('Hamiltonian DOS', size=TITLE_SIZE)
    axs[0, 0].set_xlabel('P(E)', size=LABEL_SIZE)
    axs[0, 0].set_ylabel('Energy (E)', size=LABEL_SIZE)
    axs[0, 0].grid(True)
    
    # Eigenvalues vs Index
    H_indices = np.arange(len(H_eigval))
    axs[0, 1].scatter(H_indices, np.sort(H_eigval), s=1, c=COLORS['H'], alpha=0.5)
    axs[0, 1].set_title('Hamiltonian Eigenvalues', size=TITLE_SIZE)
    axs[0, 1].set_xlabel('Index', size=LABEL_SIZE)
    axs[0, 1].set_ylabel('Energy (E)', size=LABEL_SIZE)
    axs[0, 1].grid(True)

# Row 0, Col 2: H IPR
if has_H_IPR and n_cols > 2:
    H_IPR = data['H_IPR'][disorder_idx, realization_idx]
    H_sort_idx = np.argsort(data['H_eigval'][disorder_idx, realization_idx]) if has_H_eigval else np.arange(len(H_IPR))
    
    axs[0, 2].scatter(np.arange(len(H_IPR)), H_IPR[H_sort_idx], s=1, c=COLORS['H'], alpha=0.5)
    axs[0, 2].set_title('Hamiltonian IPR', size=TITLE_SIZE)
    axs[0, 2].set_xlabel('Index', size=LABEL_SIZE)
    axs[0, 2].set_ylabel('IPR', size=LABEL_SIZE)
    axs[0, 2].grid(True)
    
    avg_H_IPR = np.mean(H_IPR)
    axs[0, 2].text(0.95, 0.95, f'mean={avg_H_IPR:.4f}', transform=axs[0, 2].transAxes,
                   fontsize=14, verticalalignment='top', horizontalalignment='right',
                   bbox=ANNOTATION_PROPS)

# Row 1: Spectral Localizer
if has_SL_eigval:
    SL_eigval = data['SL_eigval'][disorder_idx, realization_idx]
    
    # DOS histogram
    axs[1, 0].hist(SL_eigval, bins=100, density=True, orientation='horizontal',
                   color=COLORS['SL'], alpha=0.8)
    axs[1, 0].set_title('Spectral Localiser DOS', size=TITLE_SIZE)
    axs[1, 0].set_xlabel('P(E)', size=LABEL_SIZE)
    axs[1, 0].set_ylabel('Eigenvalue', size=LABEL_SIZE)
    axs[1, 0].grid(True)
    
    # Eigenvalues vs Index
    SL_indices = np.arange(len(SL_eigval))
    axs[1, 1].scatter(SL_indices, np.sort(SL_eigval), s=1, c=COLORS['SL'], alpha=0.5)
    axs[1, 1].set_title('Spectral Localiser Eigenvalues', size=TITLE_SIZE)
    axs[1, 1].set_xlabel('Index', size=LABEL_SIZE)
    axs[1, 1].set_ylabel('Eigenvalue', size=LABEL_SIZE)
    axs[1, 1].grid(True)

# Row 1, Col 2: SL IPR
if has_SL_IPR and n_cols > 2:
    SL_IPR = data['SL_IPR'][disorder_idx, realization_idx]
    SL_sort_idx = np.argsort(data['SL_eigval'][disorder_idx, realization_idx]) if has_SL_eigval else np.arange(len(SL_IPR))
    
    axs[1, 2].scatter(np.arange(len(SL_IPR)), SL_IPR[SL_sort_idx], s=1, c=COLORS['SL'], alpha=0.5)
    axs[1, 2].set_title('Spectral Localiser IPR', size=TITLE_SIZE)
    axs[1, 2].set_xlabel('Index', size=LABEL_SIZE)
    axs[1, 2].set_ylabel('IPR', size=LABEL_SIZE)
    axs[1, 2].grid(True)
    
    avg_SL_IPR = np.mean(SL_IPR)
    axs[1, 2].text(0.95, 0.95, f'mean={avg_SL_IPR:.4f}', transform=axs[1, 2].transAxes,
                   fontsize=14, verticalalignment='top', horizontalalignment='right',
                   bbox=ANNOTATION_PROPS)

fig.suptitle(f'DOS and IPR at Disorder W={W:.2f}, L={L}', fontsize=SUPTITLE_SIZE)
plt.show()

## 6. Figure 2: Energy-Resolved IPR

Shows IPR vs Energy to visualize the mobility edge.

In [ ]:
# Select disorder indices to plot (low, mid, high)
disorder_indices = [0, disorder_resolution // 2, disorder_resolution - 1]
n_disorders = len(disorder_indices)

print(f"Plotting for disorder values: {[disorder_values[i] for i in disorder_indices]}")

In [ ]:
if 'H_IPR' not in data and 'SL_IPR' not in data:
    print("No IPR data available. Skipping this plot.")
else:
    fig, axs = plt.subplots(2, n_disorders, figsize=(6*n_disorders, 10), constrained_layout=True)
    
    if n_disorders == 1:
        axs = axs.reshape(2, 1)
    
    for col, d_idx in enumerate(disorder_indices):
        W = disorder_values[d_idx]
        
        # Aggregate over realizations
        if 'H_eigval' in data and 'H_IPR' in data:
            H_eigval_all = data['H_eigval'][d_idx, :, :].flatten()
            H_IPR_all = data['H_IPR'][d_idx, :, :].flatten()
            
            # Apply percentile cutoff
            H_cutoff = np.percentile(H_IPR_all, IPR_PERCENTILE_CUTOFF)
            H_mask = H_IPR_all <= H_cutoff
            
            print(f"W={W:.1f} H: IPR range [{H_IPR_all.min():.6f}, {H_IPR_all.max():.6f}], cutoff={H_cutoff:.6f}")
            
            axs[0, col].scatter(H_eigval_all[H_mask], H_IPR_all[H_mask], s=0.5, c=COLORS['H'], alpha=0.3)
            axs[0, col].set_title(f'H: W={W:.1f}', size=TITLE_SIZE)
            axs[0, col].set_xlabel('Energy (E)', size=LABEL_SIZE)
            axs[0, col].set_ylabel('IPR', size=LABEL_SIZE)
            axs[0, col].grid(True)
        
        if 'SL_eigval' in data and 'SL_IPR' in data:
            SL_eigval_all = data['SL_eigval'][d_idx, :, :].flatten()
            SL_IPR_all = data['SL_IPR'][d_idx, :, :].flatten()
            
            # Apply percentile cutoff
            SL_cutoff = np.percentile(SL_IPR_all, IPR_PERCENTILE_CUTOFF)
            SL_mask = SL_IPR_all <= SL_cutoff
            
            print(f"W={W:.1f} SL: IPR range [{SL_IPR_all.min():.6f}, {SL_IPR_all.max():.6f}], cutoff={SL_cutoff:.6f}")
            
            axs[1, col].scatter(SL_eigval_all[SL_mask], SL_IPR_all[SL_mask], s=0.5, c=COLORS['SL'], alpha=0.3)
            axs[1, col].set_title(f'SL: W={W:.1f}', size=TITLE_SIZE)
            axs[1, col].set_xlabel('Eigenvalue', size=LABEL_SIZE)
            axs[1, col].set_ylabel('IPR', size=LABEL_SIZE)
            axs[1, col].grid(True)
    
    fig.suptitle(f'Energy-Resolved IPR (bottom {IPR_PERCENTILE_CUTOFF}%), L={L}', fontsize=SUPTITLE_SIZE)
    plt.show()

## 7. Figure 3: Mobility Edge Trajectory

Shows how the mobility edge changes with disorder strength.

In [ ]:
if 'H_eigval' not in data or 'H_IPR' not in data:
    print("Need both H_eigval and H_IPR to compute mobility edge. Skipping.")
else:
    n_disorder = len(disorder_values)
    n_real = data['H_eigval'].shape[1]
    
    H_Ec_lower = np.zeros((n_disorder, n_real))
    H_Ec_upper = np.zeros((n_disorder, n_real))
    
    for d_idx in range(n_disorder):
        for r_idx in range(n_real):
            H_eigval = data['H_eigval'][d_idx, r_idx]
            H_IPR = data['H_IPR'][d_idx, r_idx]
            
            Ec_l, Ec_u, _, _ = extract_mobility_edge(H_eigval, H_IPR)
            H_Ec_lower[d_idx, r_idx] = Ec_l
            H_Ec_upper[d_idx, r_idx] = Ec_u
    
    # Mean and std across realizations
    Ec_lower_mean = H_Ec_lower.mean(axis=1)
    Ec_lower_std = H_Ec_lower.std(axis=1)
    Ec_upper_mean = H_Ec_upper.mean(axis=1)
    Ec_upper_std = H_Ec_upper.std(axis=1)
    
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), constrained_layout=True)
    
    axs[0].errorbar(disorder_values, Ec_lower_mean, yerr=Ec_lower_std,
                    label=f'L={L}', color=COLORS['H'], marker='o', capsize=3)
    axs[0].set_title('Lower Mobility Edge (E < 0)', size=TITLE_SIZE)
    axs[0].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
    axs[0].set_ylabel('E_c (lower)', size=LABEL_SIZE)
    axs[0].legend()
    axs[0].grid(True)
    
    axs[1].errorbar(disorder_values, Ec_upper_mean, yerr=Ec_upper_std,
                    label=f'L={L}', color=COLORS['H'], marker='o', capsize=3)
    axs[1].set_title('Upper Mobility Edge (E > 0)', size=TITLE_SIZE)
    axs[1].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
    axs[1].set_ylabel('E_c (upper)', size=LABEL_SIZE)
    axs[1].legend()
    axs[1].grid(True)
    
    fig.suptitle('Mobility Edge Trajectory vs Disorder', fontsize=SUPTITLE_SIZE)
    plt.show()

## 8. Figure 4: Filtered r/z Statistics

Spectral statistics filtered by the mobility edge energy window.

In [ ]:
if 'H_eigval' not in data or 'H_IPR' not in data:
    print("Need both H_eigval and H_IPR for filtered statistics. Skipping.")
else:
    n_disorder = len(disorder_values)
    n_real = data['H_eigval'].shape[1]
    
    H_r_vals = np.zeros((n_disorder, n_real))
    H_z_vals = np.zeros((n_disorder, n_real))
    SL_r_vals = np.zeros((n_disorder, n_real)) if 'SL_eigval' in data else None
    SL_z_vals = np.zeros((n_disorder, n_real)) if 'SL_eigval' in data else None
    
    for d_idx in range(n_disorder):
        for r_idx in range(n_real):
            H_eigval = data['H_eigval'][d_idx, r_idx]
            H_IPR = data['H_IPR'][d_idx, r_idx]
            
            # Extract mobility edge
            Ec_l, Ec_u, _, _ = extract_mobility_edge(H_eigval, H_IPR)
            
            # Filter H eigenvalues
            H_filtered = filter_eigenvalues_by_energy(H_eigval, Ec_l, Ec_u)
            
            if len(H_filtered) > 5:
                H_r_vals[d_idx, r_idx] = calculate_r(H_filtered)
                H_z_vals[d_idx, r_idx] = calculate_z(H_filtered)
            else:
                H_r_vals[d_idx, r_idx] = np.nan
                H_z_vals[d_idx, r_idx] = np.nan
            
            # Filter SL eigenvalues using H's mobility edge
            if 'SL_eigval' in data:
                SL_eigval = data['SL_eigval'][d_idx, r_idx]
                SL_filtered = filter_eigenvalues_by_energy(SL_eigval, Ec_l, Ec_u)
                
                if len(SL_filtered) > 5:
                    SL_r_vals[d_idx, r_idx] = calculate_r(SL_filtered)
                    SL_z_vals[d_idx, r_idx] = calculate_z(SL_filtered)
                else:
                    SL_r_vals[d_idx, r_idx] = np.nan
                    SL_z_vals[d_idx, r_idx] = np.nan
    
    # Compute means and standard errors
    H_r_mean = np.nanmean(H_r_vals, axis=1)
    H_r_std = np.nanstd(H_r_vals, axis=1) / np.sqrt(n_real)
    H_z_mean = np.nanmean(H_z_vals, axis=1)
    H_z_std = np.nanstd(H_z_vals, axis=1) / np.sqrt(n_real)
    
    fig, axs = plt.subplots(2, 2, figsize=FIGSIZE_2x2, constrained_layout=True)
    
    # H r-statistic
    axs[0, 0].errorbar(disorder_values, H_r_mean, yerr=H_r_std,
                       label=f'L={L}', marker='o', capsize=3, color=COLORS['H'])
    axs[0, 0].axhline(y=GOE_R, color='red', linestyle='--', label='GOE', alpha=0.7)
    axs[0, 0].axhline(y=POISSON_R, color='green', linestyle='--', label='Poisson', alpha=0.7)
    axs[0, 0].set_title('Hamiltonian <r> (filtered)', size=TITLE_SIZE)
    axs[0, 0].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
    axs[0, 0].set_ylabel('<r>', size=LABEL_SIZE)
    axs[0, 0].legend()
    axs[0, 0].grid(True)
    
    # H z-statistic
    axs[0, 1].errorbar(disorder_values, H_z_mean, yerr=H_z_std,
                       label=f'L={L}', marker='o', capsize=3, color=COLORS['H'])
    axs[0, 1].axhline(y=GOE_Z, color='red', linestyle='--', label='GOE', alpha=0.7)
    axs[0, 1].axhline(y=POISSON_Z, color='green', linestyle='--', label='Poisson', alpha=0.7)
    axs[0, 1].set_title('Hamiltonian <z> (filtered)', size=TITLE_SIZE)
    axs[0, 1].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
    axs[0, 1].set_ylabel('<z>', size=LABEL_SIZE)
    axs[0, 1].legend()
    axs[0, 1].grid(True)
    
    # SL statistics (if available)
    if SL_r_vals is not None:
        SL_r_mean = np.nanmean(SL_r_vals, axis=1)
        SL_r_std = np.nanstd(SL_r_vals, axis=1) / np.sqrt(n_real)
        SL_z_mean = np.nanmean(SL_z_vals, axis=1)
        SL_z_std = np.nanstd(SL_z_vals, axis=1) / np.sqrt(n_real)
        
        axs[1, 0].errorbar(disorder_values, SL_r_mean, yerr=SL_r_std,
                           label=f'L={L}', marker='o', capsize=3, color=COLORS['SL'])
        axs[1, 0].axhline(y=GOE_R, color='red', linestyle='--', label='GOE', alpha=0.7)
        axs[1, 0].axhline(y=POISSON_R, color='green', linestyle='--', label='Poisson', alpha=0.7)
        axs[1, 0].set_title('Spectral Localizer <r> (filtered)', size=TITLE_SIZE)
        axs[1, 0].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
        axs[1, 0].set_ylabel('<r>', size=LABEL_SIZE)
        axs[1, 0].legend()
        axs[1, 0].grid(True)
        
        axs[1, 1].errorbar(disorder_values, SL_z_mean, yerr=SL_z_std,
                           label=f'L={L}', marker='o', capsize=3, color=COLORS['SL'])
        axs[1, 1].axhline(y=GOE_Z, color='red', linestyle='--', label='GOE', alpha=0.7)
        axs[1, 1].axhline(y=POISSON_Z, color='green', linestyle='--', label='Poisson', alpha=0.7)
        axs[1, 1].set_title('Spectral Localizer <z> (filtered)', size=TITLE_SIZE)
        axs[1, 1].set_xlabel('Disorder Strength W', size=LABEL_SIZE)
        axs[1, 1].set_ylabel('<z>', size=LABEL_SIZE)
        axs[1, 1].legend()
        axs[1, 1].grid(True)
    
    fig.suptitle(f'Filtered Spectral Statistics (|E| < E_c), L={L}', fontsize=SUPTITLE_SIZE)
    plt.show()